In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600, chunk_overlap=100, separator="\n"
)

loader = UnstructuredFileLoader("./files/1984.docx")

docs = loader.load_and_split(text_splitter=splitter)

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")


embedder = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)
vectorstore = FAISS.from_documents(docs, cache_embeddings)

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(temperature=0.1)

retriver = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. 
            Return any relevant text verbatim.
            ------
            {context}
                                                    """,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return (
        "\n\n".join(
            map_doc_chain.invoke(
                {"context": doc.page_content, "question": question}
            ).content
            for doc in documents
        ),
    )


map_chain = {
    "documents": retriver,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)


final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    Given the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    ------
    {context}
    """,
        ),
        (
            "human",
            "{question}",
        ),
    ]
)


chain = (
    {
        "context": map_chain,
        "question": RunnablePassthrough(),
    }
    | final_prompt
    | llm
)

chain.invoke("Where does winston go to work?")


# len(results)

AIMessage(content='Winston goes to work at the Ministry of Truth.')